### File description
Create a dataset to use together with manually annotated data; use GPT4 to create explanations for the yes/no annotations.

### Notes about using GPT4 for this task
Remarks made from analyzing 20 random samples of the dataset: rows [273, 211, 222,  77, 275, 140, 135, 124, 256,  85,  19, 190, 142, 156, 272, 111, 191,  97, 154,  24]

- Explanation may go beyond only the principle in question (eg emotions mentions amends; wanting to better manage anger).
- Discussion around that other principle might not be correct (eg 124)
- When something is not mentioned in either input story or statement, it tends to explain other things
- 24: Statement says "when an altercation arose between one of my friends and a stranger". But acc to story, there was no altercation - just "this guy said something very rude to my friend".

All-in-all GPT-4 was correct with quite good explanations 90% of the time (18/20) (counting the amends which was wrong about emotions as good...)

##### Remarks about individual samples
- "273, Good. Gets main point, and also more (for another principle)"
- "211, Good enough. Not easy to explain why something is coherent, but does well enough. No quotes.",
- "222, Good. No quotes, but there is 'no misplaced info', so that is not needed either",
- "77, Good. Acknowledges that input has no emotions, but statement has loads.",
- "275, Good. Sees plain mistake in statement (no record, which is false)",
- "140, Good. Explains emotions in story and statement, and how they cohere.",
- "135, Good. Quotes the main reason for the satisfied principle.",
- "124, OK. No amends anywhere, BUT says emotions principle is also good, although this is not the case...",
- "256, Good. Discusses amends although it is in neither story.",
- "85, Perfect. Concise and 100percent correct. ",
- "19, Good. Sees the most important breach, but also goes slightly beyond",
- "190, Good enough. Gives one example, but there is more information that this that is unsupported.",
- "142, Good. (coherence...)",
- "156, Good. Points to several important quotes that exist both in statement and story.",
- "272, Good. Explains in general what is similar, but is missing quotes / more specific examples.",
- "111, No. The not-coherent part is hitting someone unintentionally, but GPT4 does not catch this.",
- "191, Good. Sees concrete examples - but also misses one bad quote.",
- "97, Good. Explains what relevant info is there and what irrelevant info is excluded.",
- "154, Good. (age is broken)",
- "24, No. Disagrees with the given answer!:O But answer is not obvious..."

## Creating the actual data

In [ ]:
from key import test_key, miri_key
from info import principles, whatCR, rewriteR, yesnoCR_turned
import numpy as np
import openai
import time
import pandas as pd
from datasets import load_dataset

openai.api_key = miri_key

yesnoCR = yesnoCR_turned

In [6]:
data_path = "/home/vsb29/rds/hpc-work/project/data_file1.json"
data = load_dataset("json", data_files=data_path)
len(data['train']['output'])

Found cached dataset json (/home/vsb29/.cache/huggingface/datasets/json/default-332cd82ced9b110c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

281

In [14]:
gpt4_prompt = "You will be presented with an input story, a statement, a question about the input story or statement, and the correct answer to the question. Please provide an explanation for why this is the correct answer by using quotes from the input story and statement. Here are a few principles that the statement should satisfy, and they might me useful in your explanation: "
explanations = np.array([])

In [35]:
f = open("explain_GPT4", "a")
for i in range(281, 282):
    print(i)
    f.write("\n###"+str(i)+"\n")
    inp = data['train']['input'][i].split("### ")
    q = data['train']['instruction'][i].split("Please answer 'yes' or 'no'.")[0].strip()
    ans = data['train']['output'][i]
    chat_log = [{"role": "system", "content": gpt4_prompt + principles},
                {"role": "user", "content": inp[1].strip() + "\n" + inp[2] + "\nQUESTION: " + q + "\nANSWER: "+ans + "\nEXPLANATION: "}]
    explanation = answ = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=chat_log
        )
    explanations = np.append(explanations, explanation)
    f.write(explanations[i]["choices"][0]["message"]["content"])
f.close()
    

281


IndexError: list index out of range

In [36]:
f.close()

In [33]:
print(len(explanations))
print(len(data['train']['output']))

280
281


In [ ]:
new_data = {'input': data['train']['input'], 'question': data['train']['instruction'], 'answer': data['train']['output'], 'explanation': explanations}
data_df = pd.DataFrame.from_dict(new_data)
json_str = data_df.to_json('data_explanations1.json', orient = 'records')

In [ ]:
f = open("explain_GPT4", "a")
for i in range(len(data['train']['output'])):
    f.write(explanations[i]["choices"][0]["message"]["content"])
f.close()